In [3]:
#project code name: "Sleepless in Seattle" 
#project objective: Find the expected returns of taking levearge (margin trade) on crypto assets

In [4]:
from datetime import datetime
import time 
import json 
import pandas as pd 
import requests 
import math
import matplotlib.pyplot as plt
import numpy as np

In [5]:
#Function that takes time(int) as a parameter and returns year, month, and day in array 
def time_converter(input_time):
    year = int(str(input_time)[:4])
    month = int(str(input_time)[4:6])
    day = int(str(input_time)[6:8])
    time_list = [year, month, day]
    return(time_list)

In [6]:
#print(time_converter(20200101)), returns [2020, 1, 1]

In [340]:
# The function splits the data into 1500 slots and combines into one frame in a given start time and end time. 
# parameters: start period, end period, resolution, market name(pair) 
# return: date, high, low fully binded 
# resolution is the window length in seconds (15, 60, 300, 900, 3600, 14400, 86400, or any multiple of 86400 up to 30*86400)

def data_binder(start_time, end_time, resolution, market_name):
    #Select time 
    st_list = time_converter(start_time)
    st_time_convert = datetime(st_list[0], st_list[1], st_list[2]).timestamp()
    ed_list = time_converter(end_time) 
    ed_time_convert = datetime(ed_list[0], ed_list[1], ed_list[2]).timestamp()
    
    total_time = (ed_time_convert - st_time_convert) 
    
    num_slot = ((total_time) / resolution)
    num_df = math.ceil(num_slot /1500)  #start from 0 which will naturally round up, since the maximum number of a request is 1500 
    
    endpoint_url = 'https://ftx.com/api/markets'
    last_data = requests.get(f'{endpoint_url}/{market_name}/candles?resolution={resolution}&start_time={st_time_convert}&end_time={ed_time_convert}').json()
    last_data_df = pd.DataFrame(last_data['result'])
    
    #iterate and bind all fragmented dataframes 
    page_df = 1 
    last_end_time = (last_data_df.time[0] / 1000) - resolution 
    all_eth_price_history = last_data_df
    while(page_df < num_df):
        next_data = requests.get(f'{endpoint_url}/{market_name}/candles?resolution={resolution}&start_time={st_time_convert}&end_time={last_end_time}').json()
        next_df = pd.DataFrame(next_data['result'])
        combine_frame = [next_df, all_eth_price_history]
        all_eth_price_history = pd.concat(combine_frame, ignore_index = True) #merge 
        last_end_time = (next_df.time[0] / 1000) - resolution 
        page_df += 1
    return(all_eth_price_history[['startTime', 'time', 'high', 'low', 'volume']])

In [ ]:
#Create sample data sets and store in csv file

In [388]:
bull_sample1 = data_binder(20210719, 20210730, 300, 'ETH/USD') #resolution: 5 min 

In [431]:
bull_sample1.to_csv('bull_sample1.csv')

In [342]:
bull_sample2 = data_binder(20220313, 20220328, 300, 'ETH/USD') 

In [43]:
bull_sample2.to_csv('bull_sample2.csv')

In [343]:
bear_sample1 = data_binder(20211202, 20220106, 300, 'ETH/USD') 

In [45]:
bear_sample1.to_csv('bear_sample1.csv')

In [344]:
bear_sample2 =  data_binder(20220403, 20220509, 300, 'ETH/USD') 

In [47]:
bear_sample2.to_csv('bear_sample2.csv')

In [345]:
sideways_sample = data_binder(20220921, 20221018, 300, 'ETH/USD') 

In [49]:
sideways_sample.to_csv('sideways.csv')

In [50]:
#The function calculates the liqduiation price 
#parameters: current_price (float or int), leverage_cosntant(float or int), position_type(string)
#returns: the liquidation price (=tirgger price, int)

def liquidation_cal(open_price, leverage_constant, position_type):
    if (position_type == 'long'):
        liquidation_ratio = round(100 / leverage_constant, 4)/100
        trigger_price = round((open_price - (open_price * liquidation_ratio)),2)
    if (position_type == 'short'): 
        liquidation_ratio = round(100 / leverage_constant, 4)/100
        trigger_price = round((open_price + (open_price * liquidation_ratio)),2)
    return(trigger_price)

In [51]:
#Check liquidation_cal 
print(liquidation_cal(1000.0, 10, 'long')) #900.0
print(liquidation_cal(1000, 10, 'short')) #1100.0

900.0
1100.0


In [52]:
#The function calculates the profit made from an open position 
#parameters: max_price (float or int), open_price(float or int), leverage_constant(float or int)
#returns: the profit in percentage (float, rounded 2 decimals)

def profit_percent_cal(max_price, open_price, leverage_constant):
        profit = abs(max_price - open_price)
        profit_percent = round(((profit / open_price) * leverage_constant), 4) * 100
        return (round(profit_percent, 2))

In [377]:
# The function returns the dataframe that includes liquidation period, check of liquidation, maximum profit, percentage of 
# the maximum profit, and volume at the time of liquidation for the long positions. 

# parameters: 
# price_type: Define which price type to use to open a position (string, 'highest' or 'lowest')
# leverage_constant: Define leverage amount 
# resolution: Define candle size (in seconds)
# data_set: The sample data set that is used

# return: 
# The function returns final_result, which is a dataframe consist of 5 following arrays. 
# liquidation_period: Array for all liquidation periods  
# liquidation_checker: Array checking if the position is liquidated
# max_profit: Array for all maximum profit before liquidation happens 
# max_profit_prcnt: Array for all maximum profit percentage 
# volume: Array that records volume at the time of liquidation 


def leverage_checker_long(price_type, leverage_constant, resolution, data_set): 
    #Define price type (lowest from candle, highest from candle) 
    if(price_type == 'lowest'): 
        open_price_type = 'low'
    elif(price_type == 'highest'): 
        open_price_type  = 'high'
    else: 
        return("Invalid input data, select lowest or highest")
    
    
    data_duration = int((data_set.time[1] - data_set.time[0]) / 1000)
    if(int(resolution) != data_duration):
        return("Invalid input data, check if the resolution matches")
    
    #Define ouput variables 
    liquidation_period = [] #Time taken untill full liquidation 
    liquidation_checker = [] #0 if not liquidated, 1 if liquidated 
    max_profit = [] #Maximum profit
    max_profit_prcnt = [] #Maximum profit percentage  
    volume = [] #Volume at the liquidation 

    num_position = data_set.shape[0] #Number of open positions (each candle)

    #loop through all slots 
    for i in range (0, num_position - 1): 
        open_position = data_set[open_price_type][i]
 
        #Check liquidation
        liquidation_status = 0
        max_profit_price = open_position
        position_type = 'long'
        
        #Find trigger price 
        trigger_price = liquidation_cal(open_position, leverage_constant, position_type)
        for j in range(i+1, num_position):  
            if (max_profit_price < data_set.high[j]):
                max_profit_price = data_set.high[j]
                max_profit_amount = round(abs(open_position - max_profit_price),3)

            #Check if hit the trigger 
            check_price = data_set['low'][j] 

            if (check_price <= trigger_price): 
                #update output variables 
                liquidation_period.append((j * resolution) / 60) #record time taken (minute)
                
                liquidation_status = 1 #check liquidation
                liquidation_checker.append(liquidation_status)
                
                max_profit.append((max_profit_amount))
                margin_made = profit_percent_cal(max_profit_price, open_position, leverage_constant)
                max_profit_prcnt.append((margin_made))
                
                volume.append(data_set.volume[j])
                break    
                
            elif((j == (num_position - 1)) and (liquidation_status == 0)):
                liquidation_period.append(((j * resolution) / 60)) #record time taken (minute)
                liquidation_checker.append(liquidation_status)
                max_profit.append(max_profit_amount)
                margin_made = profit_percent_cal(max_profit_price, open_position, leverage_constant)
                max_profit_prcnt.append((margin_made))
                volume.append(data_set.volume[j])
                break 
        
    final_result = {'lq period':liquidation_period, 'lq check':liquidation_checker, 'max profit':max_profit,'max %': max_profit_prcnt, 'volume':volume}    
    return(pd.DataFrame(final_result))

In [372]:
# The function returns the dataframe that includes liquidation period, check of liquidation, maximum profit, percentage of 
# the maximum profit, and volume at the time of liquidation for short positions. 

# parameters: 
# price_type: Define which price type to use to open a position (string, 'highest' or 'lowest')
# leverage_constant: Define leverage amount 
# resolution: Define candle size (in seconds)
# data_set: The sample data set that is used

# return: 
# The function returns final_result, which is a dataframe consist of 5 following arrays. 
# liquidation_period: Array for all liquidation periods  
# liquidation_checker: Array checking if the position is liquidated
# max_profit: Array for all maximum profit before liquidation happens 
# max_profit_prcnt: Array for all maximum profit percentage 
# volume: Array that records volume at the time of liquidation 


def leverage_checker_short(price_type, leverage_constant, resolution, data_set): 
    #Define price type (lowest from candle, highest from candle) 
    if(price_type == 'lowest'): 
        open_price_type = 'low'
    elif(price_type == 'highest'): 
        open_price_type  = 'high'
    else: 
        return("Invalid input data, select lowest or highest")
    
    #Check duration matches with the data set
    data_duration = int((data_set.time[1] - data_set.time[0]) / 1000)
    if(int(resolution) != data_duration):
        #print(resolution, data_duration)
        return("Invalid input data, check if the resolution matches")
     
    #Define ouput variables 
    liquidation_period = [] #Time taken untill full liquidation 
    liquidation_checker = [] #0 if not liquidated, 1 if liquidated 
    max_profit = [] #Maximum profit
    max_profit_prcnt = [] #Mximum profit percentage  
    volume = [] #Volume at the liquidation 

    num_position = data_set.shape[0] #Number of open positions (each candle)

    #loop through all slots 
    for i in range (0, num_position): 
        open_position = data_set[open_price_type][i]
 
        #Check liquidation
        liquidation_status = 0
        max_profit_price = open_position
        max_profit_amount = 0 
        position_type = 'short'
        
        #Find trigger price 
        trigger_price = liquidation_cal(open_position, leverage_constant, position_type)
        
        for j in range(i+1, num_position):  
            if (max_profit_price < data_set.low[j]):
                max_profit_price = data_set.low[j]
                max_profit_amount = round(abs(open_position - max_profit_price),3)

            #Check if hit the trigger 
            check_price = data_set['high'][j] 
            if (check_price >= trigger_price): 
                #update output variables 
                liquidation_period.append((j * resolution) / 60) #record time taken (minute)
                
                liquidation_status = 1 #check liquidation
                liquidation_checker.append(liquidation_status)
            
                max_profit.append((max_profit_amount))
                margin_made = profit_percent_cal(max_profit_price, open_position, leverage_constant)
                max_profit_prcnt.append((margin_made))
                
                volume.append(data_set.volume[j])
                break    
                
            elif((j == (num_position - 1)) and (liquidation_status == 0)):
                liquidation_period.append(((j * resolution) / 60)) #record time taken (minute)
                liquidation_checker.append(liquidation_status)
                max_profit.append(max_profit_amount)
                margin_made = profit_percent_cal(max_profit_price, open_position, leverage_constant)
                max_profit_prcnt.append((margin_made))
                volume.append(data_set.volume[j])
                break 

    final_result = {'lq period':liquidation_period, 'lq check':liquidation_checker, 'max profit':max_profit,'max %': max_profit_prcnt, 'volume':volume}
    return(pd.DataFrame(final_result))

In [82]:
#Note that the possibility of liquidation decreases when there are less remaining slots to check. 

In [442]:
#1. Ride the Tide 
#1.1 Short in Bull vs Long in Bull (sample 1 & sample 2)
#1.2 Short in Bear vs Long in Bear (sample 1 & sample 2)
#1.3 Short in Sideways vs Long in Bull (sample 1)

#2. Impact of the Leverage 
#2.1 Compare Long in Bull (3x 15x 30x)
#2.2 Compare Short in Bear (3x, 15x, 30x)

#3. Does your trading skill matters? 
#3.1 Long in Bull (Highest vs Lowest)
#3.2 Short in Bear (Highest vs Lowest)


In [390]:
bull1_short = leverage_checker_short('highest', 3, 300, bull_sample1)

In [430]:
bull1_short.to_csv('bull1_short.csv')

In [393]:
bull2_short = leverage_checker_short('highest', 3, 300, bull_sample2)

In [429]:
bull2_short.to_csv('bull2_short.csv')

In [394]:
bull1_long = leverage_checker_long('highest', 3, 300, bull_sample1)

In [428]:
bull1_long.to_csv('bull1_long.csv')

In [432]:
bull2_long = leverage_checker_long('highest', 3, 300, bull_sample2)

In [434]:
bull2_long.to_csv('bull2_long.csv')

In [438]:
bear1_long = leverage_checker_long('highest', 3, 300, bear_sample1)

In [439]:
bear1_long.to_csv('bear1_long.csv')

In [440]:
bear1_short = leverage_checker_short('highest', 3, 300, bear_sample1)

In [441]:
bear1_short.to_csv('bear1_short.csv')

In [397]:
bear2_long = leverage_checker_long('highest', 3, 300, bear_sample2)

In [427]:
bear2_long.to_csv('bear2_long.csv')

In [436]:
bear2_short = leverage_checker_short('highest', 3, 300, bear_sample2)

In [437]:
bear2_short.to_csv('bear2_short.csv')

In [398]:
sideways_short = leverage_checker_short('highest', 3, 300, sideways_sample)

In [426]:
sideways_short.to_csv('sideways_short.csv')

In [399]:
sideways_long = leverage_checker_long('highest', 3, 300, sideways_sample)

In [425]:
sideways_long.to_csv('sideways_long.csv')

In [400]:
bull1_levg_3 = leverage_checker_long('highest', 3, 300, bull_sample1)

In [424]:
bull1_levg_3.to_csv('bull1_levg_3.csv')

In [401]:
bull1_levg_15 = leverage_checker_long('highest', 15, 300, bull_sample1)

In [423]:
bull1_levg_15.to_csv('bull1_levg_15.csv')

In [402]:
bull1_levg_30 = leverage_checker_long('highest', 30, 300, bull_sample1)

In [422]:
bull1_levg_30.to_csv('bull1_levg_30.csv')

In [406]:
bear1_levg_3 = leverage_checker_short('highest', 3, 300, bear_sample1)

In [421]:
bear1_levg_3.to_csv('bear1_levg_3.csv')

In [407]:
bear1_levg_15 = leverage_checker_short('highest', 15, 300, bear_sample1)

In [420]:
bear1_levg_15.to_csv('bear1_levg_15.csv')

In [408]:
bear1_levg_30 = leverage_checker_short('highest', 30, 300, bear_sample1)

In [419]:
bear1_levg_30.to_csv('bear1_levg_30.csv')

In [404]:
bull1_long_high = leverage_checker_short('highest', 3, 300, bull_sample1)

In [418]:
bull1_long_high.to_csv('bull1_long_high.csv')

In [405]:
bull1_long_low = leverage_checker_short('lowest', 3, 300, bull_sample1)

In [417]:
bull1_long_low.to_csv('bull1_long_low.csv')

In [410]:
bear1_short_high = leverage_checker_short('highest', 3, 300, bear_sample1)

In [415]:
bear1_short_high.to_csv('bear1_short_high.csv')

In [411]:
bear1_short_low = leverage_checker_short('lowest', 3, 300, bear_sample1)

In [416]:
bear1_short_low.to_csv('bear1_short_low.csv')